In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

2025-12-02 03:51:21.988850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764647482.293306      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764647482.374581      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
# -----------------------------
# PINN model definition (TensorFlow / Keras)
# -----------------------------
class PINN(tf.keras.Model):
    """
    Simple fully-connected network that takes time t as input
    and outputs two variables: x(t) and y(t).
    We subclass tf.keras.Model so that:
      - layers are registered as trainable variables
      - we can use model(...) like a normal Keras model
      - it integrates nicely with Keras optimizers / saving / etc.
    """
    def __init__(self):
        super(PINN, self).__init__()
        # First hidden layer: 64 units, tanh activation
        self.hidden_1 = tf.keras.layers.Dense(64, activation='tanh')
        # Second hidden layer: 64 units, tanh activation
        self.hidden_2 = tf.keras.layers.Dense(64, activation='tanh')
        # Output layer: 2 units, linear activation -> [x(t), y(t)]
        self.output_3 = tf.keras.layers.Dense(2, activation=None)

    def call(self, t):
        """
        Forward pass of the network.
        Input:
          t : shape (N, 1), time points
        Output:
          shape (N, 2), where:
            output[:, 0] ≈ x(t)
            output[:, 1] ≈ y(t)
        """
        z = self.hidden_1(t)
        z = self.hidden_2(z)
        return self.output_3(z)


# -----------------------------
# Loss function (PDE/ODE residual + initial conditions)
# -----------------------------
def loss_fn(model, t):
    """
    Compute PINN loss for the system:
        dx/dt + 2x + y = 0
        dy/dt + x + 2y = 0
    with initial conditions:
        x(0) = 1,  y(0) = 0

    Inputs:
      model : PINN instance
      t     : shape (N, 1) tensor of time points in [0, T]
    Output:
      scalar loss (tf.Tensor)
    """

    # We need derivatives of x(t) and y(t) w.r.t. t.
    # Two GradientTapes are used, each watching t, so that we can
    # compute dx/dt and dy/dt separately.
    with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
        # Tell both tapes that t is a "variable" they should differentiate w.r.t.
        tape1.watch(t)
        tape2.watch(t)

        # Forward pass: network outputs [x(t), y(t)] for all collocation points
        output = model(t)           # shape (N, 2)
        '''if output[:,0] is used, its size will be (N,) tensor instead of (N,1)
        '''
        x = output[:, 0:1]          # shape (N, 1) -> keep 2D tensor
        y = output[:, 1:2]          # shape (N, 1)

        # First-order derivatives w.r.t. t
        # dx_dt[i] ~ d x(t_i) / d t_i
        dx_dt = tape1.gradient(x, t)
        dy_dt = tape2.gradient(y, t)

        # ODE residuals (what should be zero for an exact solution)
        # From:
        #   dx/dt + 2x +  y = 0  -> residual_x = dx/dt + 2x + y
        #   dy/dt +  x + 2y = 0  -> residual_y = dy/dt + x + 2y
        res_x = dx_dt + 2.0 * x + y
        res_y = dy_dt + x + 2.0 * y

        # Initial-condition residuals at t = 0.
        # We assume that t[0] = 0, so x[0] and y[0] correspond to x(0), y(0).
        # x(0) should be 1,  y(0) should be 0.
        init_loss_x = tf.square(x[0] - 1.0)
        init_loss_y = tf.square(y[0] - 0.0)

        # PDE/ODE residual loss: mean-squared error over all collocation points
        loss_res_x = tf.reduce_mean(tf.square(res_x))
        loss_res_y = tf.reduce_mean(tf.square(res_y))

        # Total loss = ODE residuals + initial-condition penalties
        loss = loss_res_x + loss_res_y + init_loss_x + init_loss_y

    return loss


# -----------------------------
# Training loop
# -----------------------------
def train(model, t, epochs, optimizer):
    """
    Simple custom training loop.

    Inputs:
      model     : PINN instance
      t         : collocation points, shape (N, 1)
      epochs    : number of gradient descent steps
      optimizer : tf.keras optimizer (e.g., Adam)
    """
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            # Compute current loss
            loss = loss_fn(model, t)

        # Compute gradients w.r.t. all trainable parameters (weights + biases)
        grads = tape.gradient(loss, model.trainable_variables)

        # Gradient descent update
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Simple logging every 500 epochs
        if epoch % 500 == 0:
            print(f'Epoch {epoch}: Loss = {loss.numpy():.6e}')


# -----------------------------
# Create model, optimizer, and training data
# -----------------------------
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Training domain: N time points between 0 and 5.
# np.linspace(... )[:, None] creates an array of shape (N, 1).
t = tf.convert_to_tensor(np.linspace(0.0, 5.0, 100)[:, None], dtype=tf.float32)

# Train the model
train(model, t, epochs=4000, optimizer=optimizer)

# -----------------------------
# Prediction on a finer time grid
# -----------------------------
t_test = tf.convert_to_tensor(np.linspace(0.0, 5.0, 500)[:, None], dtype=tf.float32)
pred = model(t_test).numpy()       # shape (500, 2)
x_pred = pred[:, 0]                # shape (500,)
y_pred = pred[:, 1]                # shape (500,)



In [ ]:
# Analytical solutions
x_true = 0.5 * np.exp(-t_test) + 0.5 * np.exp(-3 * t_test)
y_true = -0.5 * np.exp(-t_test) + 0.5 * np.exp(-3 * t_test)


# Set up the plot with two subplots side by side
plt.figure(figsize=(12, 5))

# Plot x(t)
plt.subplot(1, 2, 1)
plt.plot(t_test, x_true, label='Analytical x(t)', color='red')
plt.plot(t_test, x_pred, '--', label='PINNs x(t)', color='blue')
plt.title(r'PINNs vs Analytical Solution $x(t)$', fontsize=14)
plt.xlabel(r'Time $t$')
plt.ylabel(r'$x(t)$')
plt.grid(True)
plt.legend(fontsize=12, loc="upper right")


# Plot y(t)
plt.subplot(1, 2, 2)
plt.plot(t_test, y_true, label='Analytical y(t)', color='red')
plt.plot(t_test, y_pred, '--', label='PINNs y(t)', color='blue')
plt.title(r'PINNs vs Analytical Solution $y(t)$', fontsize=14)
plt.xlabel(r'Time $t$')
plt.ylabel(r'$y(t)$')
plt.grid(True)
plt.legend(fontsize=12, loc="lower right")

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## -----------------------------
### PINN model definition (PyTorch)
## -----------------------------
class PINN(nn.Module):
    """
    Simple fully-connected network in PyTorch.
    Input:  t  (N, 1)
    Output: [x(t), y(t)]  (N, 2)
    """
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden_1 = nn.Linear(1, 64)   # input dim 1 -> 64
        self.hidden_2 = nn.Linear(64, 64)  # 64 -> 64
        self.output_3 = nn.Linear(64, 2)   # 64 -> 2 (x,y)

        # Activation function (tanh), we can reuse it
        self.act = nn.Tanh()

    def forward(self, t):
        """
        Forward pass.
        t : tensor of shape (N,1)
        return : shape (N,2)
        """
        z = self.act(self.hidden_1(t))
        z = self.act(self.hidden_2(z))
        return self.output_3(z)


# -----------------------------
# Loss function (same ODE system)
# -----------------------------
def loss_fn(model, t):
    """
    Compute PINN loss in PyTorch for:
        dx/dt + 2x + y = 0
        dy/dt + x + 2y = 0
    with x(0) = 1, y(0) = 0.
    """

    # We want derivatives w.r.t. t -> make sure t requires grad
    t = t.clone().detach().requires_grad_(True)  # keep shape (N,1)

    # Forward pass
    output = model(t)          # (N,2)
    x = output[:, 0:1]         # (N,1)
    y = output[:, 1:2]         # (N,1)

    # Compute dx/dt and dy/dt using autograd.grad
    # grad_outputs = ones to get element-wise derivative
    dx_dt = torch.autograd.grad(
        x, t,
        grad_outputs=torch.ones_like(x),
        create_graph=True
    )[0]                        # shape (N,1)

    dy_dt = torch.autograd.grad(
        y, t,
        grad_outputs=torch.ones_like(y),
        create_graph=True
    )[0]                        # shape (N,1)

    # ODE residuals
    res_x = dx_dt + 2.0 * x + y
    res_y = dy_dt + x + 2.0 * y

    # Mean-squared residuals over all collocation points
    loss_res_x = torch.mean(res_x**2)
    loss_res_y = torch.mean(res_y**2)

    # Initial-condition residuals (assume t[0] = 0)
    # x(0) should be 1,  y(0) should be 0
    init_loss_x = (x[0] - 1.0)**2
    init_loss_y = (y[0] - 0.0)**2

    # Total loss
    loss = loss_res_x + loss_res_y + init_loss_x + init_loss_y
    return loss


# -----------------------------
# Training loop (PyTorch)
# -----------------------------
def train(model, t, epochs, optimizer):
    """
    Custom training loop in PyTorch.
    t : collocation points (N,1) on chosen device.
    """
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        loss = loss_fn(model, t)
        loss.backward()            # backprop through network and dx/dt, dy/dt
        optimizer.step()

        if epoch % 500 == 0:
            print(f"[PyTorch] Epoch {epoch}: Loss = {loss.item():.6e}")


# -----------------------------
# Build model, optimizer, training data
# -----------------------------
model_torch = PINN().to(device)
optimizer_torch = optim.Adam(model_torch.parameters(), lr=1e-3)

# Training domain: time points between 0 and 5 (same as TF version)
t_np = np.linspace(0.0, 5.0, 100).reshape(-1, 1)
t_torch = torch.tensor(t_np, dtype=torch.float32, device=device)

# Train model
train(model_torch, t_torch, epochs=4000, optimizer=optimizer_torch)

# -----------------------------
# Prediction
# -----------------------------
model_torch.eval()
with torch.no_grad():
    t_test_np = np.linspace(0.0, 5.0, 500).reshape(-1, 1)
    t_test_torch = torch.tensor(t_test_np, dtype=torch.float32, device=device)
    pred_torch = model_torch(t_test_torch)        # (500,2)
    x_pred_t  = pred_torch[:, 0].cpu().numpy()    # (500,)
    y_pred_t  = pred_torch[:, 1].cpu().numpy()


In [ ]:
t_test = t_test_np.squeeze()   # (500,1) -> (500,)
x_pred = x_pred_t              
y_pred = y_pred_t


# Analytical solutions
x_true = 0.5 * np.exp(-t_test) + 0.5 * np.exp(-3 * t_test)
y_true = -0.5 * np.exp(-t_test) + 0.5 * np.exp(-3 * t_test)

# Set up the plot with two subplots side by side
plt.figure(figsize=(12, 5))

# Plot x(t)
plt.subplot(1, 2, 1)
plt.plot(t_test, x_true, label='Analytical x(t)', color='red')
plt.plot(t_test, x_pred, '--', label='PINNs x(t)', color='blue')
plt.title(r'PINNs vs Analytical Solution $x(t)$', fontsize=14)
plt.xlabel(r'Time $t$')
plt.ylabel(r'$x(t)$')
plt.grid(True)
plt.legend(fontsize=12, loc="upper right")


# Plot y(t)
plt.subplot(1, 2, 2)
plt.plot(t_test, y_true, label='Analytical y(t)', color='red')
plt.plot(t_test, y_pred, '--', label='PINNs y(t)', color='blue')
plt.title(r'PINNs vs Analytical Solution $y(t)$', fontsize=14)
plt.xlabel(r'Time $t$')
plt.ylabel(r'$y(t)$')
plt.grid(True)
plt.legend(fontsize=12, loc="lower right")

# Adjust layout and show the plot
plt.tight_layout()
plt.show()